In [1]:
import modules.singleton_reduction as singleton
import modules.isolate_reduction as isolate
from tqdm import tqdm
import importlib
importlib.reload(singleton)
importlib.reload(isolate)
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor
import io
import copy


In [2]:
# Load the full graph
edges = pd.read_csv("../data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"../data/nodes.csv", sep=",")
nodes_array = nodes["# index"]

G_full = nx.DiGraph()
G_full.add_edges_from(edges_array)
G_full.add_nodes_from(nodes_array)

## Full network

In [3]:
importlib.reload(singleton)
importlib.reload(isolate)

G = copy.deepcopy(G_full)
G = singleton.singleton_reduction(G)
G_full_sr = copy.deepcopy(G)
# isolatedNodes = list(nx.isolates(G))
G, isolatedNum = isolate.isolate_reduction(G)

# There is no longer a need to compute, we can simply load it from the positions file:
# import json
# data = json.load(open("positions_all_nodes_excluding_isolated.json"))
# pos = {d["node"]: (d["x"], d["y"]) for d in data}
# But if you want to:

Node count before singleton reduction:  58742
Nodes before:  52124
Nodes after:  19257


In [25]:
for i in nx.weakly_connected_components(G):
    print(i)

{32768, 32769, 32770, 3, 4, 5, 32774, 7, 32776, 9, 10, 6, 6515, 8, 15, 32786, 19, 20, 21, 22, 32791, 24, 32787, 26, 30, 31, 32, 35, 36, 37, 38, 39, 32807, 40, 41, 43, 42, 32813, 32814, 32815, 32817, 32820, 32821, 32822, 57, 32826, 59, 60, 61, 32828, 32830, 64, 32831, 63, 67, 68, 69, 71, 72, 73, 74, 6527, 76, 78, 32847, 80, 81, 82, 32851, 84, 83, 86, 32856, 32858, 52121, 32863, 32865, 98, 99, 100, 32866, 32867, 32869, 32870, 105, 32874, 107, 108, 109, 106, 111, 32879, 32880, 32882, 32883, 115, 117, 118, 119, 120, 121, 32890, 123, 122, 32884, 39101, 127, 129, 130, 131, 132, 133, 39104, 136, 137, 138, 139, 140, 141, 142, 143, 144, 32913, 146, 147, 148, 145, 150, 32914, 152, 32921, 153, 154, 149, 151, 158, 159, 160, 161, 32928, 163, 32927, 165, 166, 32929, 168, 169, 170, 171, 32932, 32941, 167, 176, 178, 32949, 182, 181, 183, 185, 32954, 186, 184, 189, 190, 32959, 192, 32958, 191, 32964, 32965, 198, 199, 200, 201, 203, 207, 208, 58658, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224,

In [26]:
import modules.sir_threaded as sir
import importlib
import copy
importlib.reload(sir)
init_infected = [9]
max_steps = 30
G_temp = copy.deepcopy(G)

# Uncomment below block if you want to only view nodes that are accessible from init_infected
G_temp = G_temp.subgraph(sir.get_accessible_sus_nodes(G_temp, init_infected)).copy()
print("Nodes to calculate: ", len(G_temp))
pos = nx.spring_layout(G_temp, seed=8020, gravity=0.75)

nodelist_total, nodecolors_total, edgecolors_total, options, infotext_total, constants_total = sir.sir_model(G_temp, G_full_sr, init_infected=init_infected, infection_rate=1, recovery_rate=0, max_steps=max_steps, doSingletonReduction=True, noticeability_rates=(0,0.01), network_type='full')

Running model...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  7569
[DEBUG] SSW:  951
[DEBUG] ST:  4992
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6823
[DEBUG] SSW:  205
[DEBUG] ST:  1916
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6725
[DEBUG] SSW:  107
[DEBUG] ST:  1169
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6711
[DEBUG] SSW:  93
[DEBUG] ST:  1010
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6700
[DEBUG] SSW:  82
[DEBUG] ST:  913
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6679
[DEBUG] SSW:  61
[DEBUG] ST:  432
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[D

In [27]:
import modules.sir_threaded as sir
import importlib
import copy
importlib.reload(sir)
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool
from functools import partial

# > Clear all pngs in graphs folder
for filename in os.listdir("graphs"):
   file_path = os.path.join("graphs", filename)
   if os.path.isfile(file_path):
      os.remove(file_path)

pos_2 = copy.deepcopy(pos)

worker = partial(
    sir.work,
    G=G_temp,
    pos=pos_2,
    nodelist_total=nodelist_total,
    nodecolors_total=nodecolors_total,
    edgecolors_total=edgecolors_total,
    infotext_total=infotext_total,
    constants_total=constants_total,
    options=options,
)

with Pool(16) as p:
    p.map(worker, range(len(nodelist_total)))

Starting graph image: 1


/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(


Starting graph image: 3
Starting graph image: 2
Starting graph image: 0
Starting graph image: 4
Saved graph_1.png!
Starting graph image: 5


/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(
/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(
/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a 

Starting graph image: 16
Starting graph image: 6
Starting graph image: 7
Saved graph_3.png!
Starting graph image: 17
Saved graph_0.png!


...

Saved graph_19.png!
Starting graph image: 12
Saved graph_20.png!
Starting graph image: 23
Saved graph_8.png!
Starting graph image: 24


/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(
/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(


Starting graph image: 13
Saved graph_9.png!
Starting graph image: 14
Saved graph_21.png!
Saved graph_10.png!
Starting graph image: 25
Saved graph_22.png!
Saved graph_11.png!
Starting graph image: 26
Saved graph_23.png!
Starting graph image: 15
Saved graph_12.png!
Starting graph image: 27
Saved graph_24.png!
Saved graph_25.png!
Saved graph_13.png!
Starting graph image: 28
Saved graph_14.png!


/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(
/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(
/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a 

Starting graph image: 29
Saved graph_26.png!
Saved graph_15.png!
Saved graph_27.png!
Starting graph image: 30
Saved graph_28.png!
Saved graph_29.png!
Saved graph_30.png!


## Exporting as GIF

In [28]:
# > Export images in graphs folder to a gif
from PIL import Image

# List of image file paths
images = sorted(os.listdir("graphs"), key=lambda x:int(x[x.find('_')+1:x.find('.')]))

# Open images and store them in a list
frames = [Image.open(f"graphs/{image}") for image in images]

# Save frames as an animated GIF
frames[0].save(
    'gifs/sir_model_full.gif',
    save_all=True,
    append_images=frames[1:],
    duration=500,
    loop=0
)

## Finding number of nodes and which nodes needed to infect the whole network

In [87]:
from modules.number_of_parent_nodes import parent_node_number
edges = pd.read_csv("../data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"../data/nodes.csv", sep=",")
nodes_array = nodes["# index"]
# With isolated nodes (no in or out degree)
infected_nodes_list_isolated = parent_node_number(edges_array, nodes_array, includeIsolatedNodes=True)
# Gathering number of nodes
print(len(infected_nodes_list_isolated))
# List of nodes
print(infected_nodes_list_isolated)

# Without isolated nodes
infected_nodes_list_no_isolated = parent_node_number(edges_array, nodes_array, includeIsolatedNodes=False)
# Gathering number of nodes
print(len(infected_nodes_list_no_isolated))
# List of nodes
print(infected_nodes_list_no_isolated)



38221
[169, 594, 81, 595, 131, 10, 596, 384, 660, 657, 661, 665, 666, 755, 15, 1091, 69, 342, 119, 160, 1766, 981, 111, 299, 817, 2462, 2463, 137, 1158, 261, 2464, 121, 147, 816, 231, 2871, 3063, 3064, 199, 3168, 3957, 892, 6, 3169, 3170, 336, 74, 57, 705, 1707, 2658, 1651, 2117, 2122, 7046, 7047, 1472, 821, 2155, 959, 2366, 7770, 2852, 1084, 7774, 7775, 7776, 7777, 1094, 7779, 1096, 7783, 7784, 7785, 2009, 24, 963, 4777, 2940, 1565, 146, 9615, 10109, 314, 84, 4467, 11918, 1093, 3041, 931, 522, 4049, 1579, 942, 312, 317, 190, 5455, 178, 3712, 1311, 1031, 2295, 1582, 1175, 1225, 2245, 139, 144, 2252, 2253, 9329, 1446, 1718, 490, 510, 2627, 22603, 22606, 5395, 17342, 22607, 20, 2428, 4180, 228, 4994, 233, 26978, 26963, 26967, 27021, 1434, 492, 2002, 324, 7073, 100, 1204, 28488, 3677, 4, 2684, 3308, 3119, 1290, 957, 432, 343, 1899, 2963, 3496, 1776, 2683, 2156, 10146, 2722, 5264, 1219, 535, 123, 5516, 9525, 484, 1307, 15540, 37153, 2339, 18951, 68, 2052, 7993, 619, 34609, 39501, 29013, 39

In [90]:
print(nx.number_weakly_connected_components(G))
print(nx.number_weakly_connected_components(G_full))

52
32919


In [ ]:
# TODO: need to add all infected nodes to simulation and run the simulation once to showcase infection of all nodes in network (this can be done at a later date right now im tired)